In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version

spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
#  get the postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.3.1.jar

--2022-01-24 23:34:50--  https://jdbc.postgresql.org/download/postgresql-42.3.1.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1015689 (992K) [application/java-archive]
Saving to: ‘postgresql-42.3.1.jar’

postgresql-42.3.1.j 100%[===================>] 991.88K  --.-KB/s    in 0.08s   

2022-01-24 23:34:51 (11.5 MB/s) - ‘postgresql-42.3.1.jar’ saved [1015689/1015689]



In [3]:
#create a spark  session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("HE Data") \
    .config("spark.driver.extraClassPath", "/content/postgresql-42.3.1.jar") \
    .getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
spark.sparkContext.addFile("https://expeditiondata.s3.us-east-2.amazonaws.com/updated_data.csv")
updated_df=spark.read.csv(SparkFiles.get('updated_data.csv'),header=True,  inferSchema=True)
updated_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+----------------+-------+----+-----------+----+-------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|highpoint_metres|success|solo|oxygen_used|died|height_metres|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+----------------+-------+----+-----------+----+-------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|              NA|      f|   f|          f|   f|         6814|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/197

In [5]:
#remove highpoint_metres column
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [6]:
dropped_hm_df = updated_df.drop("highpoint_metres")

In [7]:
dropped_hm_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|success|solo|oxygen_used|died|height_metres|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|      f|   f|          f|   f|         6814|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|               

In [8]:
#create list to filer df to include top 15 most frequented peaks
peak_list=['EVER','CHOY','AMAD','MANA','DHA1',
           'MAKA','LHOT','BARU','PUMO','ANN1','KANG',
           'HIML','ANN4','PUTH','TILI']

In [9]:
#filter df to only included climbers on peaks in top 15
filtered_df = dropped_hm_df.filter(dropped_hm_df.peak_id.isin(peak_list))

In [10]:
#row count of filtered df
filtered_df.count()

61638

In [11]:
#create column showing success by definition (reached summit and survived)
from pyspark.sql.functions import when
success_df = filtered_df.withColumn("defined_success", 
                when(((filtered_df.success == 't') & (filtered_df.died == 'f')),"TRUE") 
               .otherwise("FALSE")) 
success_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+---------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|success|solo|oxygen_used|died|height_metres|defined_success|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+---------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|      f|   f|          f|   f|         6814|          FALSE|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|  

In [12]:
#drop duplicates
success_df.dropDuplicates()
success_df.count()

61638

Removing Null Values

In [13]:
# Count the number of null values for each column
Dict_Null = {col:success_df.filter(success_df[col].isNull()).count() for col in success_df.columns}
Dict_Null


{'age': 8,
 'basecamp_date': 0,
 'defined_success': 0,
 'died': 8,
 'expedition_id': 0,
 'height_metres': 0,
 'highpoint_date': 0,
 'hired_staff': 0,
 'hired_staff_deaths': 0,
 'member_deaths': 0,
 'member_id': 8,
 'members': 0,
 'oxygen_used': 8,
 'peak_id': 0,
 'peak_name': 0,
 'season': 0,
 'sex': 8,
 'solo': 8,
 'success': 8,
 'termination_date': 0,
 'year_id': 0}

In [14]:
# drop null columns
# Drop the null columns where all values are null
success_df = success_df.dropna(how='all')

# Drop rows containing NaN values
success_df = success_df.dropna()



In [15]:
# Count the number of null values for each column
Dict_Null = {col:success_df.filter(success_df[col].isNull()).count() for col in success_df.columns}
Dict_Null


{'age': 0,
 'basecamp_date': 0,
 'defined_success': 0,
 'died': 0,
 'expedition_id': 0,
 'height_metres': 0,
 'highpoint_date': 0,
 'hired_staff': 0,
 'hired_staff_deaths': 0,
 'member_deaths': 0,
 'member_id': 0,
 'members': 0,
 'oxygen_used': 0,
 'peak_id': 0,
 'peak_name': 0,
 'season': 0,
 'sex': 0,
 'solo': 0,
 'success': 0,
 'termination_date': 0,
 'year_id': 0}

In [16]:
#row count after nulls dropped
success_df.count()
success_df.show()

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+---------------+
|expedition_id|peak_id| peak_name|year_id|season|basecamp_date|highpoint_date|termination_date|members|member_deaths|hired_staff|hired_staff_deaths|   member_id|sex|age|success|solo|oxygen_used|died|height_metres|defined_success|
+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+------------------+------------+---+---+-------+----+-----------+----+-------------+---------------+
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|      10/23/1978|      8|            0|          5|                 0|AMAD78301-01|  M| 40|      f|   f|          f|   f|         6814|          FALSE|
|    AMAD78301|   AMAD|Ama Dablam|   1978|Autumn|    10/1/1978|    10/20/1978|  

Convert to Pandas DF to  remove outliers

In [38]:
df = success_df.toPandas()
df

,expedition_id,peak_id,peak_name,year_id,season,basecamp_date,highpoint_date,termination_date,members,member_deaths,hired_staff,hired_staff_deaths,member_id,sex,age,success,solo,oxygen_used,died,height_metres,defined_success
0,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-01,M,40,f,f,f,f,6814,FALSE
1,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-02,M,41,f,f,f,f,6814,FALSE
2,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-03,M,27,f,f,f,f,6814,FALSE
3,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-04,M,40,f,f,f,f,6814,FALSE
4,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-05,M,34,f,f,f,f,6814,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61625,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-03,M,55,f,f,f,f,7246,FALSE
61626,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-04,M,65,f,f,f,f,7246,FALSE
61627,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-05,M,61,f,f,f,f,7246,FALSE
61628,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-06,M,NA,f,f,f,f,7246,FALSE


In [62]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop rows containing NaN values
df = df.dropna()

#Confirm there are now no nulls
df.isnull().sum()

#remove text  NA from ages
df  =df[df.age  != 'NA']

In [63]:
#convert age datatype to int
import plotly.express as px
df['age'] = df['age'].astype(int)


In [64]:
boxplot1 = px.box(data_frame=df,
           y="age",
           title="box plot for age",
           width = 600, height = 500)
boxplot1.show()

In [66]:
#remove outliers from age column
index = df[(df['age'] >= 65)|(df['age'] <= 12)].index
df.drop(index, inplace=True)
df['age'].describe()

count    59069.000000
mean        37.339467
std          9.735949
min         13.000000
25%         30.000000
50%         36.000000
75%         44.000000
max         64.000000
Name: age, dtype: float64

In [72]:
df['members'] = df['members'].astype(int)
#remove groups listed with 0 members
df  =df[df.members  != 0]

In [73]:
boxplot2 = px.box(data_frame=df,
           y="members",
           title="box plot for accompanying members",
           width = 600, height = 500)
boxplot2.show()

In [74]:
#remove outliers from members column
index = df[(df['members'] >= 27)|(df['members'] <= 0)].index
df.drop(index, inplace=True)
df['members'].describe()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



count    56128.000000
mean         9.311021
std          5.611799
min          1.000000
25%          5.000000
50%          8.000000
75%         13.000000
max         26.000000
Name: members, dtype: float64

In [77]:
boxplot3 = px.box(data_frame=df,
           y="hired_staff",
           title="box plot for hired staff",
           width = 600, height = 500)
boxplot3.show()

In [75]:
df['hired_staff'] = df['hired_staff'].astype(int)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
#remove outliers from hired_staff column
index = df[(df['hired_staff'] >= 16)|(df['hired_staff'] < 0)].index
df.drop(index, inplace=True)
df['hired_staff'].describe()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



count    52082.000000
mean         3.967782
std          3.816775
min          0.000000
25%          1.000000
50%          3.000000
75%          6.000000
max         15.000000
Name: hired_staff, dtype: float64

In [80]:
#show cleaned df
df

,expedition_id,peak_id,peak_name,year_id,season,basecamp_date,highpoint_date,termination_date,members,member_deaths,hired_staff,hired_staff_deaths,member_id,sex,age,success,solo,oxygen_used,died,height_metres,defined_success
0,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-01,M,40,f,f,f,f,6814,FALSE
1,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-02,M,41,f,f,f,f,6814,FALSE
2,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-03,M,27,f,f,f,f,6814,FALSE
3,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-04,M,40,f,f,f,f,6814,FALSE
4,AMAD78301,AMAD,Ama Dablam,1978,Autumn,10/1/1978,10/20/1978,10/23/1978,8,0,5,0,AMAD78301-05,M,34,f,f,f,f,6814,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61623,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-01,M,55,f,f,f,f,7246,FALSE
61624,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-02,M,33,f,f,f,f,7246,FALSE
61625,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-03,M,55,f,f,f,f,7246,FALSE
61627,PUTH19101,PUTH,Putha Hiunchuli,2019,Spring,5/9/2019,5/16/2019,5/17/2019,5,0,2,0,PUTH19101-05,M,61,f,f,f,f,7246,FALSE


In [81]:
#convert pandas  df to  spark df

final_clean_df=spark.createDataFrame(df) 
final_clean_df.printSchema()
final_clean_df.show()

root
 |-- expedition_id: string (nullable = true)
 |-- peak_id: string (nullable = true)
 |-- peak_name: string (nullable = true)
 |-- year_id: long (nullable = true)
 |-- season: string (nullable = true)
 |-- basecamp_date: string (nullable = true)
 |-- highpoint_date: string (nullable = true)
 |-- termination_date: string (nullable = true)
 |-- members: long (nullable = true)
 |-- member_deaths: long (nullable = true)
 |-- hired_staff: long (nullable = true)
 |-- hired_staff_deaths: long (nullable = true)
 |-- member_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: long (nullable = true)
 |-- success: string (nullable = true)
 |-- solo: string (nullable = true)
 |-- oxygen_used: string (nullable = true)
 |-- died: string (nullable = true)
 |-- height_metres: long (nullable = true)
 |-- defined_success: string (nullable = true)

+-------------+-------+----------+-------+------+-------------+--------------+----------------+-------+-------------+-----------+----

In [82]:
# Configure settings for RDS

server_name= "jdbc:postgresql://group-1.c08lganpj8oa.us-east-2.rds.amazonaws.com:5432/postgres"




In [83]:
#write data  frame to active
final_clean_df.write.jdbc(server_name,'final_clean',  mode='append',properties={
    "user":'root',
    "password":'Group1!!',
    "driver": 'org.postgresql.Driver'
})